In [5]:
# -*- coding: utf-8 -*-
from CSVtoPD import NANDCommandLoader


In [6]:
TestData = NANDCommandLoader(csv_path="./WL_01.csv")

In [7]:
TestData.load_csv()
TestData.show_head()

[INFO] CSV 파일이 성공적으로 로드되었습니다: ./WL_01.csv
   index  time      CMD     OPT_1    OPT_2    OPT_3 OPT_4 OPT_5 OPT_6 OPT_7  OPT_8  TOP_9  CH  BANK  DESCRIPTION
0      1   100  CMD(70)  DATA(80)      NaN      NaN   NaN   NaN   NaN   NaN    NaN    NaN   0     0  Read Status
1      2   200  CMD(00)   ADD(00)  ADD(01)  ADD(02)   NaN   NaN   NaN   NaN    NaN    NaN   0     0        ERASE
2      3   300  CMD(70)  DATA(80)      NaN      NaN   NaN   NaN   NaN   NaN    NaN    NaN   0     0  Read Status
3      4   400  CMD(DA)       NaN      NaN      NaN   NaN   NaN   NaN   NaN    NaN    NaN   0     0   SLC Enable
4      5   600  CMD(70)  DATA(81)      NaN      NaN   NaN   NaN   NaN   NaN    NaN    NaN   0     0  Read Status


In [10]:
import json
import math
json_data = TestData.df.to_dict(orient='records')
#print(json.dumps(json_data, indent=4 ))
# nan 값을 None으로 변환하여 json 변환 시 필드가 없는 것으로 처리

def remove_nan(d):
    return {k: v for k, v in d.items() if not (isinstance(v, float) and math.isnan(v))}

json_data_no_nan = [remove_nan(item) for item in json_data]

# JSON 파일로 저장
with open('./WL_01.json', 'w', encoding='utf-8') as f:
    json.dump(json_data_no_nan, f, indent=4, ensure_ascii=False)

print(f"[INFO] JSON 파일로 저장되었습니다: {'./WL_01.json'}")

[INFO] JSON 파일로 저장되었습니다: ./WL_01.json


In [ ]:
data = json.load(open('./WL_01.json', 'r', encoding='utf-8'))

with open('./WL_01.txt', 'w', encoding='utf-8') as f:
    for item in data:
        f.write(f"*-{item['index']} : {item['DESCRIPTION']} ----------* \n")
        # OPT_1 ~ OPT_9 값이 nan이더라도 NOP 라인 출력
        opt_keys = ['OPT_1', 'OPT_2', 'OPT_3', 'OPT_4', 'OPT_5', 'OPT_6', 'OPT_7', 'OPT_8', 'TOP_9']
        opts = [str(item.get(k)) for k in opt_keys if item.get(k) is not None]
        f.write(f"NOP {item['CMD']} {''.join(opts)} TS15 \n")

        # OPT_1 ~ OPT_9 값이 CMD인 경우
        for i, opt in enumerate(opts, start=1):
            if opt == item['CMD']:
                f.write(f"OPT_{i} {item['CMD']} {''.join(opts)} TS15 \n")
        f.write("\n")

        # OPT_1 ~ OPT_9 값이 ADD인 경우
        for i, opt in enumerate(opts, start=1):
            if opt == 'ADD':
                f.write(f"OPT_{i} {item['ADD']} {''.join(opts)} TS15 \n")
        f.write("\n")   

In [ ]:
data = json.load(open('./WL_01.json', 'r', encoding='utf-8'))

with open('./WL_01.txt', 'w', encoding='utf-8') as f:
    for item in data:
        f.write(f"*-{item['index']} : {item['DESCRIPTION']} ----------* \n")
        # NOP CMD TS15
        f.write(f"NOP {item['CMD']} TS15\n")
        # NOP OPT_1 TS15, NOP OPT_2 TS15, ...
        opt_keys = ['OPT_1', 'OPT_2', 'OPT_3', 'OPT_4', 'OPT_5', 'OPT_6', 'OPT_7', 'OPT_8', 'TOP_9']
        for k in opt_keys:
            if k in item and item[k] is not None:
                f.write(f"NOP {item[k]} TS15\n")
        f.write("\n")

In [11]:
import json
import math

json_data = TestData.df.to_dict(orient='records')

def remove_nan(d):
    return {k: v for k, v in d.items() if not (isinstance(v, float) and math.isnan(v))}

def convert_row(row):
    row = remove_nan(row)
    # CMD 값 추출
    cmd = row.get('CMD')
    # OPT_1~OPT_9, TOP_9 추출 및 변환
    opt_keys = ['OPT_1', 'OPT_2', 'OPT_3', 'OPT_4', 'OPT_5', 'OPT_6', 'OPT_7', 'OPT_8', 'TOP_9']
    opts = []
    for k in opt_keys:
        if k in row and row[k] is not None:
            # TYPE 추출 (예: ADD(00) → TYPE: ADD, VALUE: 00)
            val = str(row[k])
            if '(' in val and ')' in val:
                type_, value = val.split('(')
                value = value.rstrip(')')
                opts.append({"TYPE": type_, "VALUE": value})
            else:
                opts.append({"TYPE": val, "VALUE": ""})
    # 새 구조로 반환
    new_row = {
        "index": row.get("index"),
        "DESCRIPTION": row.get("DESCRIPTION"),
        "CMD": cmd,
        "OPT": opts
    }
    return new_row

json_data_structured = [convert_row(item) for item in json_data]

with open('./WL_01_ver01.json', 'w', encoding='utf-8') as f:
    json.dump(json_data_structured, f, indent=4, ensure_ascii=False)

print(f"[INFO] JSON 파일로 저장되었습니다: {'./WL_01_ver01.json'}")

[INFO] JSON 파일로 저장되었습니다: ./WL_01_ver01.json


In [13]:
import json
import math

json_data = TestData.df.to_dict(orient='records')

def remove_nan(d):
    return {k: v for k, v in d.items() if not (isinstance(v, float) and math.isnan(v))}

def extract_code(val):
    """CMD(00) → 00, ADD(0A) → 0A 등에서 괄호 안 값만 추출, 없으면 원본 반환"""
    val = str(val)
    if '(' in val and ')' in val:
        code = val.split('(')[1].rstrip(')')
        return code.upper()
    return val.upper()

def convert_row(row):
    row = remove_nan(row)
    # CMD 값 추출 (CMD(00) → 00)
    cmd = row.get('CMD')
    cmd_val = extract_code(cmd) if cmd else ""
    # OPT_1~OPT_9, TOP_9 추출 및 변환
    opt_keys = ['OPT_1', 'OPT_2', 'OPT_3', 'OPT_4', 'OPT_5', 'OPT_6', 'OPT_7', 'OPT_8', 'TOP_9']
    opts = []
    for k in opt_keys:
        if k in row and row[k] is not None:
            val = str(row[k])
            if '(' in val and ')' in val:
                type_, value = val.split('(')
                value = value.rstrip(')')
                # VALUE를 항상 16진수 대문자 문자열로
                try:
                    value_hex = f"{int(value, 16):02X}"
                except Exception:
                    value_hex = value.upper()
                opts.append({"TYPE": type_.strip(), "VALUE": value_hex})
            else:
                opts.append({"TYPE": val.strip(), "VALUE": ""})
    # 새 구조로 반환
    new_row = {
        "index": row.get("index"),
        "DESCRIPTION": row.get("DESCRIPTION"),
        "CMD": cmd_val,
        "OPT": opts
    }
    return new_row

json_data_structured = [convert_row(item) for item in json_data]

with open('./WL_01_ver01.json', 'w', encoding='utf-8') as f:
    json.dump(json_data_structured, f, indent=4, ensure_ascii=False)

print(f"[INFO] JSON 파일로 저장되었습니다: {'./WL_01_ver01.json'}")

[INFO] JSON 파일로 저장되었습니다: ./WL_01_ver01.json


In [16]:
data = json.load(open('./WL_01_ver01.json', 'r', encoding='utf-8'))

with open('./WL_01.txt', 'w', encoding='utf-8') as f:
    for item in data:
        # 제목 라인을 30글자로 맞추기 위해 '-' 개수 자동 조절
        desc = f"*-{item['index']} : {item['DESCRIPTION']} "
        dash_count = 30 - len(desc)
        if dash_count < 0:
            dash_count = 0
        f.write(desc + '-' * dash_count + '* \n')
        # CMD 항목
        if 'CMD' in item and item['CMD']:
            f.write(f"NOP CMD TP<#{item['CMD']} TS15\n")
        # OPT 항목
        if 'OPT' in item and isinstance(item['OPT'], list):
            for opt in item['OPT']:
                type_ = opt.get('TYPE', '')
                value = opt.get('VALUE', '')
                if type_ and value:
                    f.write(f"NOP {type_} TP<#{value} TS15\n")
                elif type_:
                    f.write(f"NOP {type_} TS15\n")
        

In [11]:
from PDtoJSON import NANDDataConverter

converter = NANDDataConverter(TestData.df)
converter.convert_and_save('./WL_01_final.json')
print(f"[INFO] 최종 JSON 파일로 저장되었습니다: {'./WL_01_final.json'}")


[INFO] JSON 파일로 저장되었습니다: ./WL_01_final.json
[INFO] 최종 JSON 파일로 저장되었습니다: ./WL_01_final.json


In [12]:
from JSONtoTEXT import NANDTextGenerator
text_generator = NANDTextGenerator('./WL_01_final.json')
text_generator.generate_and_save('./WL_01_final.txt')
print(f"[INFO] 최종 텍스트 파일로 저장되었습니다: {'./WL_01_final.txt'}")

[INFO] JSON 파일이 로드되었습니다: ./WL_01_final.json
[INFO] 텍스트 파일로 저장되었습니다: ./WL_01_final.txt
[INFO] 최종 텍스트 파일로 저장되었습니다: ./WL_01_final.txt
